In [1]:
from sklearn.datasets import fetch_openml
from sklearn.utils import check_random_state
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch

In [2]:
X, y = fetch_openml("mnist_784", version=1, return_X_y=True)

/Users/hwang-giseong/anaconda3/envs/ai/lib/python3.11/site-packages/sklearn/datasets/_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [4]:
single_sample = X.loc[0, :].values.reshape(28, 28)

In [3]:
train_sampels = 60000

x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=train_sampels)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

In [6]:
class ConvolutionalLayer:
    def __init__(self, num_kernels, kernel_shape):
        self.num_kernels = num_kernels
        self.kernel_shape = kernel_shape
        self.k = self.kernel_shape[0]
        
        self.kernel_theta = torch.randn(self.num_kernels, self.kernel_shape[0], self.kernel_shape[1])
        
    def slider(self, image):
        h, w = image.shape
        for h_idx in range(h - (self.k -1)):
            for w_idx in range(w - (self.k - 1)):
                single_slide_area = image[h_idx: (h_idx + self.k), w_idx:(w_idx + self.k)]
    
    def forward(self, images):
        assert single_sample.dim() == 2
        
        _, w =  images.shape
        p = 0
        o = (w - self.k) + 1
        print("Padding shape: ", p)
        print("Output shape: ", o)
        
        output = torch.zeros((o, o, self.num_kernels))
        
        for single_slide_area, h_idx, w_idx in self.slider(images):
            if h_idx == 0 and w_idx == 0:
                print('Region shape: \t', list(single_slide_area.shape))
                print('Kernel shape: \t', list(self.kernel_theta.shape))
                print('Single Slide: \t', list(output[h_idx, w_idx].shape))
            
            output[h_idx, w_idx] = torch.sum(single_slide_area * self.kernel_theta, axis=(1, 2))
        output = 1. / (1. + torch.exp(-output))
        
        return output